# Null 가능 및 기본값
- Milvus에서는 기본 필드를 제외한 스칼라 필드에 대해 nullable 속성 및 기본값을 설정할 수 있음
- nullable=True로 표시된 필드의 경우 데이터를 삽입할 때 해당 필드를 건너뛰거나 직접 null 값으로 설정하면 시스템 오류 없이 null로 처리

참교자료: https://milvus.io/docs/ko/nullable-and-default.md

## 제한

- 기본 피드를 제외한 스칼라 필드만 기본값 및 null 가능 속성을 지원
- JSON 및 배열 필드는 기본값 지원 x
- 기본값 또는 null 가능 속성은 컬렉션 생성 중에만 구성할 수 있으며 이후에는 수정할 수 없음
- nullable 속성이 활성화된 스칼라 필드는 그룹화 검색에서 group_by_field로 사용할 수 없음
- nullable로 표시된 필드는 파티션 키로 사용할 수 없음
- nullable 속성이 활성화된 스칼라 필드에 인덱스를 생성할 때 null 값은 인덱스에서 제외
- JSON 및 배열 필드: IS NULL 또는 IS NOT NULL 연산자를 사용하여 JSON 또는 ARRAY 필드를 필터링하는 경우, 이러한 연산자는 열 수준에서 작동하므로 전체 JSON 객체 또는 배열이 null인지 여부만 평가

## Null 가능 속성
- nullable 속성을 사용하면 컬렉션에 null 값을 저장할 수 있어 알 수 없는 데이터를 처리할 때 유연성을 제공

### null 가능 속성 설정하기
- 컬렉션을 만들 때 nullable=True을 사용하여 null 가능 필드를 정의(기본값 False)
- 다음 예제에서는 my_collection라는 이름의 컬렉션을 만들고 age 필드를 nullable로 설정

In [3]:
from pymilvus import MilvusClient, DataType

client = MilvusClient(uri='http://localhost:19530')

schema = client.create_schema(
    auto_id=False,
    enable_dynamic_schema=True,
)

schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=5)
schema.add_field(field_name="age", datatype=DataType.INT64, nullable=True) # Nullable field

# Set index params
index_params = client.prepare_index_params()
index_params.add_index(field_name="vector", index_type="AUTOINDEX", metric_type="L2")

if client.has_collection("my_collection"):
    client.drop_collection("my_collection")

# Create collection
client.create_collection(collection_name="my_collection", schema=schema, index_params=index_params)

## 엔티티 삽입
- null 가능 필드에 데이터를 삽입할 때는 null을 삽입하거나 이 필드를 직접 생략

In [4]:
data = [
    {"id": 1, "vector": [0.1, 0.2, 0.3, 0.4, 0.5], "age": 30},
    {"id": 2, "vector": [0.2, 0.3, 0.4, 0.5, 0.6], "age": None},
    {"id": 3, "vector": [0.3, 0.4, 0.5, 0.6, 0.7]}
]

client.insert(collection_name="my_collection", data=data)

{'insert_count': 3, 'ids': [1, 2, 3], 'cost': 0}

## 널값으로 검색 및 쿼리
- search 메서드를 사용할 때 필드에 null 값이 포함된 경우 검색 결과는 해당 필드를 null로 반환

In [6]:
res = client.search(
    collection_name="my_collection",
    data=[[0.1, 0.2, 0.4, 0.3, 0.128]],
    limit=2,
    search_params={"params": {"nprobe": 16}},
    output_fields=["id", "age"]
)

print(res)

data: [[{'id': 1, 'distance': 0.15838398039340973, 'entity': {'age': 30, 'id': 1}}, {'id': 2, 'distance': 0.28278401494026184, 'entity': {'age': None, 'id': 2}}]]


- query 메서드를 스칼라 필터링에 사용하는 경우, null 값에 대한 필터링 결과는 모두 거짓이며, 선택되지 않음

In [7]:
results = client.query(
    collection_name="my_collection",
    filter="age >= 0",
    output_fields=["id", "age"]
)

In [8]:
print(results)

data: ["{'id': 1, 'age': 30}"]


- null 값을 가진 엔티티를 반환하려면 다음과 같이 스칼라 필터링 조건 없이 쿼리를 진행

In [9]:
null_results = client.query(
    collection_name="my_collection",
    filter="",
    output_fields=["id", "age"],
    limit=10
)

In [10]:
null_results

data: ["{'id': 1, 'age': 30}", "{'id': 2, 'age': None}", "{'id': 3, 'age': None}"]

## 기본값
- 스칼라 필드에 할당된 사전 설정값, 삽입 중에 기본값이 있는 필드에 값을 제공하지 않으면 시스템에서 자동으로 기본값을 사용

### 기본값 설정
- 컬렉션을 만들 때 default_value 매개변수를 사용하여 필드의 기본값을 정의할 수 있음

In [31]:
schema = client.create_schema(
    auto_id=False,
    enable_dynamic_schema=True,
)

schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=4)
schema.add_field(field_name="age", datatype=DataType.INT64, default_value=18)
schema.add_field(field_name="status", datatype=DataType.VARCHAR, default_value="active", max_length=10)

index_params = client.prepare_index_params()
index_params.add_index(field_name="vector", index_type="AUTOINDEX", metric_type="L2")

if client.has_collection("my_collection"):
    client.drop_collection("my_collection")

client.create_collection(collection_name="my_collection", schema=schema, index_params=index_params)

## 엔티티 삽입
- 데이터를 삽입할 때 기본값이 있는 필드를 생략하거나 해당 값을 null로 설정하면 시스템에서 기본값을 사용